# Elictricity

Data source: http://www.eia.gov/electricity/data/eia861/

Set up the environment and load libraries.

In [1]:
import re
import requests
import zipfile

import numpy as np
import pandas as pd

pd.set_option('float_format', '{:6.2f}'.format)

Download the webpage with data links.

In [2]:
url = 'http://www.eia.gov/electricity/data/eia861/'
response = requests.get(url)

Find all zip files mentioned on the webpage with the help of regular expressions.

In [3]:
fnames = re.findall('zip\/(.*?\.zip)', str(response.content))
print(fnames)

['f8612014er.zip', 'f8612013.zip', 'f8612012.zip', 'f86111.zip', 'f86110.zip', 'f86109.zip', 'f86108.zip', 'f86107.zip', 'f86106.zip', 'f86105.zip', 'f86104.zip', 'f86103.zip', 'f86102.zip', 'f86101.zip', 'f86100.zip', 'f86199.zip', 'f86198.zip', 'f86197.zip', 'f86196.zip', 'f86195.zip', 'f86194.zip', 'f86193.zip', 'f86192.zip', 'f86191.zip', 'f86190.zip']


Find all years from zip file names.

In [4]:
years = [re.search('\d+', x).group()[-2:] for x in fnames]
years = ['20' + x if int(x) < 50 else '19' + x for x in years]
years = [int(x) for x in years]
print(years)

[2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990]


In [5]:
files = dict(zip(years, fnames))
print(files)

{1990: 'f86190.zip', 1991: 'f86191.zip', 1992: 'f86192.zip', 1993: 'f86193.zip', 1994: 'f86194.zip', 1995: 'f86195.zip', 1996: 'f86196.zip', 1997: 'f86197.zip', 1998: 'f86198.zip', 1999: 'f86199.zip', 2000: 'f86100.zip', 2001: 'f86101.zip', 2002: 'f86102.zip', 2003: 'f86103.zip', 2004: 'f86104.zip', 2005: 'f86105.zip', 2006: 'f86106.zip', 2007: 'f86107.zip', 2008: 'f86108.zip', 2009: 'f86109.zip', 2010: 'f86110.zip', 2011: 'f86111.zip', 2012: 'f8612012.zip', 2013: 'f8612013.zip', 2014: 'f8612014er.zip'}


Download all zip files to the disk.

In [6]:
path = '../data/Electricity/'

for year, fname in files.items():
    response = requests.get(url + 'zip/' + fname)
    with open(path + str(year) + '.zip', "wb") as file:
        file.write(response.content)

Load year only 'Distributed_Generation_2013.xls' into Pandas DataFrame.

In [7]:
generation = {2013: 'Distributed_Generation_2013.xls',
              2012: 'distributed_generation_2012.xls'}

year = 2013
zf = zipfile.ZipFile(path + str(year) + '.zip')
raw = pd.read_excel(zf.open(generation[year]),
                    index_col=list(range(4)), na_values=['.'],
                    sheetname=0, header=1).dropna()

print(raw.dtypes)
print(raw[['Number of Generators', 'Total Capacity in MWs']].head())

Number of Generators            int64
Total Capacity in MWs         float64
Capacity Back-up Only         float64
Capacity Utility Owned        float64
Estimated or Actual Data       object
Internal Combustion Engine    float64
Combusiton Turbine            float64
Steam Turbine                 float64
HydroElectric                 float64
Wind                          float64
Photovoltaic                  float64
Storage                       float64
Other                         float64
Total                         float64
Estimated or Actual Data.1     object
dtype: object
                                                              Number of Generators  \
Data Year Utility Number Utility Name                  State                         
2013      276            Alcorn County Elec Power Assn MS                        9   
          392            Alpena Power Co               MI                       29   
          1529           Beltrami Electric Coop, Inc   MN               

Read two years of data and concatenate them into one dataset.

In [8]:
raw_frames = []
for year, name in generation.items():
    zf = zipfile.ZipFile(path + str(year) + '.zip')
    raw_frames.append(pd.read_excel(zf.open(generation[year]),
                                    index_col=list(range(4)), na_values=['.'],
                                    sheetname=0, header=1))

distributed_generation = pd.concat(raw_frames)

print(distributed_generation.shape)
print(distributed_generation.dtypes)

(513, 15)
Number of Generators            int64
Total Capacity in MWs         float64
Capacity Back-up Only         float64
Capacity Utility Owned        float64
Estimated or Actual Data       object
Internal Combustion Engine    float64
Combusiton Turbine            float64
Steam Turbine                 float64
HydroElectric                 float64
Wind                          float64
Photovoltaic                  float64
Storage                       float64
Other                         float64
Total                         float64
Estimated or Actual Data.1     object
dtype: object


How many unique years, utility companies, and states do we have in the dataset?

In [9]:
for level in distributed_generation.index.names:
    print(level, distributed_generation.index.get_level_values(level).nunique())

Data Year 2
Utility Number 283
Utility Name 297
State 50


Aggregate capacity by year and state.

In [10]:
fun = {'Capacity': np.sum, 'Companies': pd.Series.count}
capacity = distributed_generation.groupby(level=['Data Year', 'State'])['Total'].agg(fun)
capacity.sort_index(by='Capacity', inplace=True)

print(capacity.head())
print(capacity.tail())

                 Companies  Capacity
Data Year State                     
2013      TN          1.00      0.00
2012      TN          1.00      0.20
          MT          2.00      0.20
          WV          2.00      0.40
2013      ND          1.00      0.90
                 Companies  Capacity
Data Year State                     
2012      FL         10.00    254.00
2013      MA          7.00    292.60
          FL          9.00    294.90
2012      CA         16.00    483.20
2013      CA         16.00    621.60


Compute means for each state.

In [11]:
capacity_state = capacity.groupby(level='State').mean()
capacity_state['Mean generation per company'] = capacity_state.eval('Capacity / Companies')
capacity_state.sort_index(by='Mean generation per company', inplace=True)

print(capacity_state.head())
print(capacity_state.tail())
print(capacity_state.describe())

       Companies  Capacity  Mean generation per company
State                                                  
TN          1.00      0.10                         0.10
WV          2.00      0.40                         0.20
SD          6.00      1.45                         0.24
KS         10.50      8.50                         0.81
NV          1.00      1.00                         1.00
       Companies  Capacity  Mean generation per company
State                                                  
FL          9.50    274.45                        28.89
CT          1.00     29.75                        29.75
CA         16.00    552.40                        34.53
MA          5.50    202.30                        36.78
NJ          1.50     99.90                        66.60
       Companies  Capacity  Mean generation per company
count      50.00     50.00                        50.00
mean        5.12     47.55                         9.68
std         4.71     90.17                      

Compute Herfindahl index (https://en.wikipedia.org/wiki/Herfindahl_index) for each state. For that we first need to compute market share of each company.

In [12]:
fun = {'Capacity': np.sum, 'Companies': pd.Series.count}
share = distributed_generation.groupby(level=['Data Year', 'State'])['Total'].apply(lambda x: x / x.sum())

print(share.head())
print(share.describe())

Data Year  Utility Number  Utility Name              State
2012       97              Adams Electric Coop       IL        0.05
           122             Village of Arcade         NY        0.01
           123             City of Adel- (GA)        GA        0.00
           232             City of Albemarle - (NC)  NC        0.01
           392             Alpena Power Co           MI        0.05
dtype: float64
count   508.00
mean      0.19
std       0.30
min       0.00
25%       0.01
50%       0.04
75%       0.20
max       1.00
dtype: float64


And now compute the index itself.

In [13]:
def compute_hf(shares):
    if len(shares) == 1:
        return 1.
    else:
        return ((shares**2).sum() - 1/len(shares)) / (1 - 1/len(shares))

hfindex = share.groupby(level=['Data Year', 'State']).apply(compute_hf)
hfindex = hfindex.groupby(level='State').mean().order()

print(hfindex.describe())
print(hfindex.head())
print(hfindex.tail())

count    50.00
mean      0.56
std       0.30
min       0.05
25%       0.33
50%       0.57
75%       0.79
max       1.00
dtype: float64
State
KS     0.05
UT     0.07
NC     0.15
KY     0.16
CO     0.18
dtype: float64
State
TN     1.00
DC     1.00
CT     1.00
AR     1.00
NV     1.00
dtype: float64


Compute capacity for eacy year and state by technology.

In [14]:
techs = raw.columns[5:-2]
technology = distributed_generation.groupby(level=['Data Year', 'State'])[techs].sum().fillna(0)

print(technology.describe().ix['mean'].order())

Storage                        0.18
Steam Turbine                  0.91
Combusiton Turbine             1.99
HydroElectric                  2.26
Wind                           3.72
Other                          4.54
Internal Combustion Engine    17.04
Photovoltaic                  17.76
Name: mean, dtype: float64


Convert capacity to shares.

In [15]:
technology = technology.apply(lambda x: x / x.sum(), axis=1)

print(technology.describe().ix['mean'].order())

Storage                        0.00
Combusiton Turbine             0.03
Steam Turbine                  0.05
Other                          0.06
Wind                           0.07
HydroElectric                  0.12
Photovoltaic                   0.30
Internal Combustion Engine     0.37
Name: mean, dtype: float64


Compute Herfindahl index over technology types.

In [16]:
tech_hfindex = technology.apply(compute_hf, axis=1)
tech_hfindex = tech_hfindex.groupby(level='State').mean().order()

print(tech_hfindex.head())
print(tech_hfindex.tail())
print(tech_hfindex.describe())

State
MI     0.14
IL     0.18
LA     0.24
NY     0.29
CT     0.29
dtype: float64
State
NJ     0.97
MD     0.98
NV     1.00
AL     1.00
WV     1.00
dtype: float64
count    50.00
mean      0.61
std       0.25
min       0.14
25%       0.43
50%       0.56
75%       0.86
max       1.00
dtype: float64


In [17]:
df = technology.xs(tech_hfindex.index[0], level='State')
print(df)

           Internal Combustion Engine  Combusiton Turbine  Steam Turbine  \
Data Year                                                                  
2012                             0.10                0.31           0.08   
2013                             0.09                0.28           0.10   

           HydroElectric   Wind  Photovoltaic  Storage  Other  
Data Year                                                      
2012                0.05   0.35          0.10     0.00   0.01  
2013                0.05   0.37          0.10     0.00   0.01  


Load efficiency data for 2013. Note that it does not have 'Ownership Type' column. We can load another dataset that has this column and merge them.

In [18]:
efficiency = {2013: 'Energy_Efficiency_2013.xls',
              2012: 'dsm_2012.xls'}

year = 2013
zf = zipfile.ZipFile(path + str(year) + '.zip')
eff2013 = pd.read_excel(zf.open(efficiency[year]),
                        index_col=list(range(4)), na_values=['.'],
                        skipfooter=1, sheetname=0, header=2)

zf = zipfile.ZipFile(path + str(year) + '.zip')
utility_data2013 = pd.read_excel(zf.open('Utility_Data_2013.xls'),
                               index_col=list(range(5)), na_values=['.'],
                               sheetname=0, header=1)

eff2013 = pd.merge(eff2013, utility_data2013.reset_index('Ownership Type'),
                  left_index=True, right_index=True)

eff2013.set_index('Ownership Type', append=True, inplace=True)

eff2013 = eff2013.loc[:, 'Total']

print(eff2013.head())

Data Year  Utility Number  Utility Name                   State  Ownership Type
2013       122             Village of Arcade - (NY)       NY     Municipal          324.00
           189             PowerSouth Energy Cooperative  AL     Cooperative       4428.00
           195             Alabama Power Co               AL     Investor Owned   14657.00
           207             City of Alameda                CA     Municipal         3076.00
           295             City of Alexandria - (MN)      MN     Municipal         1867.00
Name: Total, dtype: float64


Load efficiency data for 2012.

In [19]:
year = 2012
zf = zipfile.ZipFile(path + str(year) + '.zip')
eff2012 = pd.read_excel(zf.open(efficiency[year]),
                        index_col=list(range(5)), na_values=['.'],
                        sheetname=0, header=2)

eff2012 = eff2012.loc[:, 'Total']

print(eff2012.head())

Data Year  Utility Number  Utility Name                    State  Ownership  
2012       84              A & N Electric Coop             VA     Cooperative   100.00
           97              Adams Electric Coop             IL     Cooperative     0.00
           108             Adams-Columbia Electric Coop    WI     Cooperative     0.00
           118             Adams Rural Electric Coop, Inc  OH     Cooperative     0.00
           122             Village of Arcade               NY     Municipal       0.00
Name: Total, dtype: float64


Concatenate two datasets.

In [20]:
efficiency = pd.concat([eff2012, eff2013])
efficiency.name = 'Savings'

print(efficiency.index.names)
print(efficiency.head())

['Data Year', 'Utility Number', 'Utility Name', 'State', 'Ownership']
Data Year  Utility Number  Utility Name                    State  Ownership  
2012       84              A & N Electric Coop             VA     Cooperative   100.00
           97              Adams Electric Coop             IL     Cooperative     0.00
           108             Adams-Columbia Electric Coop    WI     Cooperative     0.00
           118             Adams Rural Electric Coop, Inc  OH     Cooperative     0.00
           122             Village of Arcade               NY     Municipal       0.00
Name: Savings, dtype: float64


Compute total savings for each year and state.

In [21]:
efficiency_agg = efficiency.groupby(level=['State', 'Ownership']).sum()

print(efficiency_agg.head())

State  Ownership            
AK     Cooperative             3655.00
       Investor Owned             0.00
       Municipal                   nan
       Political Subdivision       nan
AL     Cooperative             9907.00
Name: Savings, dtype: float64


Compute total capacity for each year and state. Convert MWs to MWh.

In [22]:
distr_generation = distributed_generation.groupby(level='State').sum()
distr_generation = distr_generation.loc[:, 'Total Capacity in MWs'] * 60**2
distr_generation.name = 'Capacity'

print(distr_generation.head())

State
AK    182520.00
AL    182520.00
AR     16920.00
AZ    263880.00
CA   3977280.00
Name: Capacity, dtype: float64


Merge two datasets.

In [23]:
norm_eff = pd.merge(pd.DataFrame(efficiency_agg).reset_index('Ownership'),
                    pd.DataFrame(distr_generation), left_index=True, right_index=True)

norm_eff.set_index('Ownership', append=True, inplace=True)

print(norm_eff.head())

                             Savings  Capacity
State Ownership                               
AK    Cooperative            3655.00 182520.00
      Investor Owned            0.00 182520.00
      Municipal                  nan 182520.00
      Political Subdivision      nan 182520.00
AL    Cooperative            9907.00 182520.00


Compute savings relative to capacity.

In [24]:
norm_eff['Relative Savings'] = norm_eff.eval('Savings / Capacity')

print(norm_eff.head())

                             Savings  Capacity  Relative Savings
State Ownership                                                 
AK    Cooperative            3655.00 182520.00              0.02
      Investor Owned            0.00 182520.00              0.00
      Municipal                  nan 182520.00               nan
      Political Subdivision      nan 182520.00               nan
AL    Cooperative            9907.00 182520.00              0.05


Compute average savings.

In [25]:
savings = norm_eff.groupby(level='Ownership')['Relative Savings'].mean().order()

print(savings)

Ownership
Retail Power Marketer        0.00
Municipal Mktg Authority     0.17
Cooperative                  0.24
State                        0.52
Municipal                    0.53
Political Subdivision        2.33
DSM Administrator            3.84
Investor Owned               5.96
Federal                    120.53
Name: Relative Savings, dtype: float64
